# 2DCOS Analysis Toolkit for CD
Perform complete Two-Dimensional Correlation Spectroscopy (2DCOS) analysis  
on Far-UV Circular Dichroism (CD) data, including MRE conversion, preprocessing,  
correlation map computation, peak detection, and result export.

In [ ]:
#@title Setup
#@markdown Prepare the environment for the analysis. Run this once before starting.

import os
import sys
import subprocess
from pathlib import Path

def _project_root_from_cwd() -> Path:
    root = Path.cwd().resolve()
    if root.name.lower() == "notebooks":
        root = root.parent
    return root

try:
    root = _project_root_from_cwd()
    os.chdir(root)

    pyproject = root / "pyproject.toml"
    if not pyproject.exists():
        raise FileNotFoundError(f"Setup failed: '{pyproject}' not found.")

    # Install package in editable mode (quiet-ish)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-e", "."])

    # Import public API
    from dcos_toolkit.api import (
        init_session,
        load_input_data_and_parse,
        compute_mre,
        compute_2dcos,
        visualize_session,
        package_results,
    )

    print("Setup complete.")

except subprocess.CalledProcessError as e:
    raise RuntimeError(f"Setup failed: pip install returned exit code {e.returncode}.") from e

In [ ]:
#@title Input Data
#@markdown Upload one or more CD data files in **.csv**, **.xlsx**, **.xls** or a **.zip** archive containing such files.
#@markdown
#@markdown **Expected table format:**
#@markdown - Header row: perturbation values for each spectrum (e.g., temperatures)
#@markdown - Column 1: wavelength **λ (nm)**
#@markdown - Body: CD values (**mdeg**)
#@markdown
#@markdown **JASCO:** ASCII/CSV exports are supported; CD is read from **Channel 1**.

from pathlib import Path

job_name = "my_analysis"  #@param {type:"string"}

# Fixed project layout (root-level data folder)
input_dir  = Path("data/input_cd")
output_dir = Path("data/results")

SESSION = init_session(job_name=job_name, output_dir=str(output_dir))
SESSION.input_dir = input_dir

# Ensure directories exist
SESSION.input_dir.mkdir(parents=True, exist_ok=True)
SESSION.output_dir.mkdir(parents=True, exist_ok=True)

print(f"Input folder: {SESSION.input_dir}")
print(f"Output folder: {SESSION.output_dir}")

# Parse / unzip / discover supported files
load_input_data_and_parse(SESSION, paths=[SESSION.input_dir]);

In [ ]:
#@title CD parameters for MRE calculation
#@markdown ## Formula for Mean Residue Ellipticity
#@markdown $$[\theta] = \frac{M \cdot \theta_{\text{obs}}}{10 \cdot c \cdot l \cdot n}$$
#@markdown **Where:**
#@markdown - **M** – molar mass of the peptide *(g/mol)*
#@markdown - **θ_obs** – observed ellipticity *(mdeg)*
#@markdown - **c** – concentration of the sample *(mg/mL)*
#@markdown - **l** – optical path length *(cm)*
#@markdown - **n** – number of peptide bonds *(≈ number of residues)*
#@markdown ---
#@markdown ### Parameter description
#@markdown All uploaded CD files are assumed to correspond to the **same sample**.
#@markdown The parameters below (**n, c, l, M**) are applied to **all uploaded datasets**.
#@markdown For different peptides with different physical parameters,
#@markdown please run the notebook separately for each peptide.
#@markdown
#@markdown - `residues_number` (**n**) – number of peptide bonds / residues
#@markdown - `concentration` (**c**) – peptide concentration in **mg/mL**
#@markdown - `path_length` (**l**) – cuvette optical path length in **cm**
#@markdown - `molar_mass` (**M**) – peptide molar mass in **g/mol**

molar_mass     = 1   #@param {type:"number"}
concentration  = 1   #@param {type:"number"}
path_length    = 1   #@param {type:"number"}
residues_number = 1  #@param {type:"number"}

#@markdown ---
#@markdown ### Plot generator
#@markdown If enabled, 1D spectra will be shown and saved for **all** datasets.
generate_mre_plot = True   #@param {type:"boolean"}
use_mre_for_plot  = False  #@param {type:"boolean"}
#@markdown - If `True`, plots show calculated **MRE** [θ].
#@markdown - If `False`, plots show **original CD** in mdeg.



compute_mre(
    SESSION,
    residues_number=residues_number,
    concentration_mg_ml=concentration,
    path_length_mm=path_length,
    molar_mass_g_mol=molar_mass,
    generate_plot=generate_mre_plot,
    use_mre_for_plot=use_mre_for_plot,
)
print("✅ MRE calculation done.")

In [ ]:
#@title **2D-COS Calculation**
#@markdown Compute **synchronous (Φ)** and **asynchronous (Ψ)** 2D correlation maps for all parsed datasets.
#@markdown ---
#@markdown **Input signal**
use_mre_for_2dcos = True  #@param {type:"boolean"}
#@markdown - If `True`, use **MRE** (requires the MRE step to be completed first).
#@markdown - If `False`, use **raw CD**.

#@markdown ---
#@markdown **Reference spectrum (subtraction mode)**
reference_type = "mean"  #@param ["mean", "first", "last", "none"]
#@markdown - **mean**: subtract the mean spectrum (recommended default)
#@markdown - **first**: subtract the first spectrum
#@markdown - **last**: subtract the last spectrum
#@markdown - **none**: no subtraction (uses the original spectra)


compute_2dcos(
    SESSION,
    use_mre_for_2dcos=use_mre_for_2dcos,
    reference_type=reference_type,
)
print("✅ 2DCOS calculation done.")

In [ ]:
#@title **2D-COS Visualization**
#@markdown Generate high-resolution **combined figures** (Φ + Ψ) with optional peak annotations.
#@markdown ---
#@markdown ### Plot settings
colormap = "jet"  #@param ["jet", "bwr", "coolwarm", "viridis", "seismic"]
lambda_min = 0.0      #@param {type:"number"}
lambda_max = 0.0      #@param {type:"number"}
#@markdown - If `lambda_min >= lambda_max`, the full wavelength range is used.

#@markdown ---
#@markdown ### Peak mirroring
mark_mirror_peaks = True  #@param {type:"boolean"}
#@markdown - If `True`, peaks are marked on both symmetric halves of the map (mirrored across the diagonal).
#@markdown - If `False`, peaks are marked only on one half; mirrored positions are not shown.

#@markdown ---
#@markdown ### Peak annotation (how many peaks to label)
#@markdown Select the number of most prominent peaks in each category (0–5). Use small values for readability.
#@markdown
#@markdown **Synchronous Φ**
n_sync_diag_peaks = "1"          #@param ["0","1","2","3","4","5"]
#@markdown - Diagonal (auto-peaks)

n_sync_cross_max_peaks = "1"     #@param ["0","1","2","3","4","5"]
#@markdown - Positive cross-peaks (Φ > 0)

n_sync_cross_min_peaks = "1"     #@param ["0","1","2","3","4","5"]
#@markdown - Negative cross-peaks (Φ < 0)

#@markdown **Asynchronous Ψ**
n_async_cross_max_peaks = "1"    #@param ["0","1","2","3","4","5"]
#@markdown - Positive cross-peaks (Ψ > 0)

n_async_cross_min_peaks = "1"    #@param ["0","1","2","3","4","5"]
#@markdown - Negative cross-peaks (Ψ < 0)


visualize_session(
    SESSION,
    colormap=colormap,
    lambda_min=lambda_min,
    lambda_max=lambda_max,
    mark_mirror_peaks=mark_mirror_peaks,
    n_sync_diag_peaks=n_sync_diag_peaks,
    n_sync_cross_max_peaks=n_sync_cross_max_peaks,
    n_sync_cross_min_peaks=n_sync_cross_min_peaks,
    n_async_cross_max_peaks=n_async_cross_max_peaks,
    n_async_cross_min_peaks=n_async_cross_min_peaks,
)
print("✅ Visualization done.")

In [ ]:
#@title **Export Results**
#@markdown Package selected files into a ZIP file (and download it in Colab).

include_input_file     = True  #@param {type:"boolean"}
#@markdown Include the original input CD files.

include_mre            = True  #@param {type:"boolean"}
#@markdown - Include MRE tables.

include_mre_plot       = True  #@param {type:"boolean"}
#@markdown - Include 1D MRE plots.

include_2dcos          = True  #@param {type:"boolean"}
#@markdown - Include 2D-COS matrices.

include_2dcos_plot     = True  #@param {type:"boolean"}
#@markdown - Include 2D-COS map plots.

try:
    zip_path = package_results(
        SESSION,
        include_input_file=include_input_file,
        include_mre=include_mre,
        include_mre_plot=include_mre_plot,
        include_2dcos=include_2dcos,
        include_2dcos_plot=include_2dcos_plot,
    )
    print(f"📦 ZIP saved at: {zip_path}")
except Exception as e:
    print(e)
    zip_path = None